In [8]:
import numpy as np

def iterative_math(condition=1, starting_param_list=None, model=2, gens=15):
    """
    Calculates the expected variance and covariance iteratively based on a given condition,
    printing each matrix at each generation.

    Args:
        condition (int): The condition number (1-5) to use for parameter selection.
        starting_param_list (dict): A dictionary of starting parameters. If None, uses default values.
        model (int): The model number to use (only model 2 is implemented here).
        gens (int): The number of generations to simulate.
    """
    if starting_param_list is None:
        starting_param_list = {
            'vg1': np.array([.5,.5, .5]),
            'vg2': np.array([.85,.85,.85]),
            'am11': np.array([0.4, 0, 0.4]),
            'am12': np.array([0.1, 0, 0.1]),
            'am21': np.array([0.05, 0, 0.05]),
            'am22': np.array([0.3, 0, 0.3]),
            'f11': np.array([0.15, 0.15,0]),
            'f12': np.array([0.1, 0.1, 0]),
            'f21': np.array([0.05, 0.05, 0]),
            'f22': np.array([0.1, 0.1, 0]),
            'Nfam': np.array([1.6e5, 1.6e5, 1.6e5]),
            'rg': np.array([.1, 0,0]),
            're': np.array([0.5, 0,0]),
            'prop.h2.latent1': np.array([0.34/0.5, 0.34/0.5, 0.34/0.5]),
            'prop.h2.latent2': np.array([0.49/0.85, 0.49/0.85, 0.49/0.85])
        }

    # --- Parameter setup for the given condition ---
    idx = condition - 1
    vg1 = starting_param_list['vg1'][idx]
    vg2 = starting_param_list['vg2'][idx]
    rg = starting_param_list['rg'][idx]
    prop_h2_latent1 = starting_param_list['prop.h2.latent1'][idx]
    prop_h2_latent2 = starting_param_list['prop.h2.latent2'][idx]
    am11 = starting_param_list['am11'][idx]
    am12 = starting_param_list['am12'][idx]
    am21 = starting_param_list['am21'][idx]
    am22 = starting_param_list['am22'][idx]
    f11 = starting_param_list['f11'][idx]
    f12 = starting_param_list['f12'][idx]
    f21 = starting_param_list['f21'][idx]
    f22 = starting_param_list['f22'][idx]
    re = starting_param_list['re'][idx]

    # --- Implied variables (t0) ---
    k2_matrix = np.array([[1, rg], [rg, 1]])

    vg_obs1 = vg1 * (1 - prop_h2_latent1)
    vg_obs2 = vg2 * (1 - prop_h2_latent2)
    d11 = np.sqrt(vg_obs1)
    d21 = 0
    d22 = np.sqrt(vg_obs2 - d21**2)
    delta_mat = np.array([[d11, 0], [d21, d22]])

    vg_lat1 = vg1 * prop_h2_latent1
    vg_lat2 = vg2 * prop_h2_latent2
    a11 = np.sqrt(vg_lat1)
    a21 = 0
    a22 = np.sqrt(vg_lat2 - a21**2)
    a_mat = np.array([[a11, 0], [a21, a22]])

    covg_mat = (delta_mat @ k2_matrix @ delta_mat.T) + (a_mat @ k2_matrix @ a_mat.T)

    ve1 = 1 - vg1
    ve2 = 1 - vg2
    cove = re * np.sqrt(ve1 * ve2)
    cove_mat = np.array([[ve1, cove], [cove, ve2]])

    COVY = covg_mat + cove_mat

    mate_cor_mat = np.array([[am11, am12], [am21, am22]])
    f_mat = np.array([[f11, f12], [f21, f22]])
    covf_mat = 2 * (f_mat @ COVY @ f_mat.T)

    # --- Initialize parameters for iteration ---
    a_t0 = a_mat
    delta_t0 = delta_mat
    j_t0 = k2_matrix * 0.5
    k_t0 = k2_matrix * 0.5
    f_t0 = f_mat
    rmate_t0 = mate_cor_mat
    covE_t0 = cove_mat

    # Lists to store matrices at each generation
    exp_VY, exp_VF, exp_mu, mate_cov = ([None] * gens for _ in range(4))
    exp_gc, exp_hc, exp_ic, exp_w, exp_v = ([None] * gens for _ in range(5))
    exp_Omega, exp_Gamma, exp_itlo, exp_itol = ([None] * gens for _ in range(4))
    exp_VGO, exp_VGL, exp_COVLO = ([None] * gens for _ in range(3))

    # Initialize matrices at t=0 (index 0)
    exp_gc[0] = exp_hc[0] = exp_ic[0] = exp_w[0] = exp_v[0] = np.zeros((2, 2))
    exp_VY[0] = COVY
    exp_VF[0] = covf_mat
    exp_mu[0] = np.linalg.inv(COVY) @ rmate_t0 @ np.linalg.inv(COVY.T)
    mate_cov[0] = COVY @ exp_mu[0] @ COVY

    print(f"=========================================")
    print(f"STARTING CONDITION {condition}")
    print(f"=========================================\n")
    
    if model == 2:
        for it in range(1, gens):
            it_prev = it - 1
            print(f"\n---------- Generation {it} ----------")

            exp_Omega[it] = (2 * delta_t0 @ exp_gc[it_prev] + delta_t0 @ k_t0 +
                             0.5 * exp_w[it_prev] + 2 * a_t0 @ exp_ic[it_prev])
            print(f"Omega ({it}):\n{exp_Omega[it]}")

            exp_Gamma[it] = (2 * a_t0 @ exp_hc[it_prev] + 2 * delta_t0 @ exp_ic[it_prev].T +
                             a_t0 @ j_t0 + 0.5 * exp_v[it_prev])
            print(f"Gamma ({it}):\n{exp_Gamma[it]}")

            exp_VY[it] = (2 * delta_t0 @ exp_Omega[it].T + 2 * a_t0 @ exp_Gamma[it].T +
                          exp_w[it_prev] @ delta_t0.T + exp_v[it_prev] @ a_t0.T +
                          exp_VF[it_prev] + covE_t0)
            print(f"VY ({it}):\n{exp_VY[it]}")
            
            vy_sqrt_diag = np.sqrt(np.diag(np.diag(exp_VY[it])))
            mate_cov[it] = vy_sqrt_diag @ rmate_t0 @ vy_sqrt_diag
            exp_mu[it] = np.linalg.inv(exp_VY[it]) @ mate_cov[it] @ np.linalg.inv(exp_VY[it].T)
            print(f"mu ({it}):\n{exp_mu[it]}")

            exp_gt = exp_Omega[it].T @ exp_mu[it] @ exp_Omega[it]
            print(f"gt ({it}):\n{exp_gt}")
            
            exp_gc[it] = 0.5 * (exp_gt + exp_gt.T)
            print(f"gc ({it}):\n{exp_gc[it]}")

            exp_ht = exp_Gamma[it].T @ exp_mu[it] @ exp_Gamma[it]
            print(f"ht ({it}):\n{exp_ht}")
            
            exp_hc[it] = 0.5 * (exp_ht + exp_ht.T)
            print(f"hc ({it}):\n{exp_hc[it]}")

            exp_w[it] = (2 * f_t0 @ exp_Omega[it] +
                         f_t0 @ exp_VY[it] @ exp_mu[it] @ exp_Omega[it] +
                         f_t0 @ exp_VY[it] @ exp_mu[it].T @ exp_Omega[it])
            print(f"w ({it}):\n{exp_w[it]}")

            exp_v[it] = (2 * f_t0 @ exp_Gamma[it] +
                         f_t0 @ exp_VY[it] @ exp_mu[it] @ exp_Gamma[it] +
                         f_t0 @ exp_VY[it] @ exp_mu[it].T @ exp_Gamma[it])
            print(f"v ({it}):\n{exp_v[it]}")

            exp_VF[it] = (2 * f_t0 @ exp_VY[it] @ f_t0.T +
                          f_t0 @ exp_VY[it] @ exp_mu[it] @ exp_VY[it] @ f_t0.T +
                          f_t0 @ exp_VY[it] @ exp_mu[it].T @ exp_VY[it] @ f_t0.T)
            print(f"VF ({it}):\n{exp_VF[it]}")

            exp_itlo[it] = exp_Gamma[it].T @ exp_mu[it] @ exp_Omega[it]
            exp_itol[it] = exp_Omega[it].T @ exp_mu[it] @ exp_Gamma[it]
            exp_ic[it] = 0.25 * (exp_itol[it] + exp_itol[it].T + exp_itlo[it] + exp_itlo[it].T)
            print(f"ic ({it}):\n{exp_ic[it]}")

            exp_VGO[it] = (2 * delta_t0 @ k_t0 @ delta_t0.T + 4 * delta_t0 @ exp_gc[it] @ delta_t0.T)
            exp_VGL[it] = (2 * a_t0 @ j_t0 @ a_t0.T + 4 * a_t0 @ exp_hc[it] @ a_t0.T)
            exp_COVLO[it] = (4 * delta_t0 @ exp_ic[it].T @ a_t0.T + 4 * a_t0 @ exp_ic[it] @ delta_t0.T)
            
    else:
        print(f"Model {model} is not implemented in this script.")
        return

    # --- Print final results ---
    print(f"\n--- Final Results for Condition {condition} after {gens-1} generations ---")
    print("True a:\n", a_mat)
    print("\nTrue delta:\n", delta_mat)
    print(f"\nr2pgs1: {delta_mat[0, 0]**2:.4f}")
    print(f"r2pgs2: {delta_mat[1, 1]**2:.4f}")
    print("-" * 45, "\n")




In [9]:
# --- Run the simulation for each condition ---
if __name__ == '__main__':
    # You can run for a single condition to keep the output manageable, for example:
    iterative_math(condition=1, model=2)
    


STARTING CONDITION 1


---------- Generation 1 ----------
Omega (1):
[[0.2  0.02]
 [0.03 0.3 ]]
Gamma (1):
[[0.29154759 0.02915476]
 [0.035      0.35      ]]
VY (1):
[[1.07710484 0.24481719]
 [0.24481719 1.02903495]]
mu (1):
[[ 0.39567858 -0.0637722 ]
 [-0.11397977  0.31142839]]
gt (1):
[[ 0.01504092  0.00049085]
 [-0.00249148  0.02712032]]
gc (1):
[[ 0.01504092 -0.00100031]
 [-0.00100031  0.02712032]]
ht (1):
[[ 0.03220037  0.00055454]
 [-0.00451749  0.03667249]]
hc (1):
[[ 0.03220037 -0.00198148]
 [-0.00198148  0.03667249]]
w (1):
[[0.09209914 0.08461725]
 [0.03603889 0.08016868]]
v (1):
[[0.13203935 0.10117916]
 [0.05028417 0.094354  ]]
VF (1):
[[0.11403913 0.06232518]
 [0.06232518 0.04077018]]
ic (1):
[[ 0.02199935 -0.00149019]
 [-0.00149019  0.03153122]]

---------- Generation 2 ----------
Omega (2):
[[0.28373773 0.05977053]
 [0.0447328  0.41677242]]
Gamma (2):
[[0.41271851 0.07624141]
 [0.05557979 0.48635596]]
VY (2):
[[1.43616882 0.42145444]
 [0.42145444 1.48594444]]
mu (2):
[[ 

In [10]:
# --- Run the simulation for VTonly condition ---
if __name__ == '__main__':
    # You can run for a single condition to keep the output manageable, for example:
    iterative_math(condition=2, model=2)
    


STARTING CONDITION 2


---------- Generation 1 ----------
Omega (1):
[[0.2 0. ]
 [0.  0.3]]
Gamma (1):
[[0.29154759 0.        ]
 [0.         0.35      ]]
VY (1):
[[1.065 0.035]
 [0.035 1.025]]
mu (1):
[[0. 0.]
 [0. 0.]]
gt (1):
[[0. 0.]
 [0. 0.]]
gc (1):
[[0. 0.]
 [0. 0.]]
ht (1):
[[0. 0.]
 [0. 0.]]
hc (1):
[[0. 0.]
 [0. 0.]]
w (1):
[[0.06 0.06]
 [0.02 0.06]]
v (1):
[[0.08746428 0.07      ]
 [0.02915476 0.07      ]]
VF (1):
[[0.070525 0.037875]
 [0.037875 0.026525]]
ic (1):
[[0. 0.]
 [0. 0.]]

---------- Generation 2 ----------
Omega (2):
[[0.23 0.03]
 [0.01 0.33]]
Gamma (2):
[[0.33527973 0.035     ]
 [0.01457738 0.385     ]]
VY (2):
[[1.220525 0.147875]
 [0.147875 1.196525]]
mu (2):
[[0. 0.]
 [0. 0.]]
gt (2):
[[0. 0.]
 [0. 0.]]
gc (2):
[[0. 0.]
 [0. 0.]]
ht (2):
[[0. 0.]
 [0. 0.]]
hc (2):
[[0. 0.]
 [0. 0.]]
w (2):
[[0.071 0.075]
 [0.025 0.069]]
v (2):
[[0.1034994  0.0875    ]
 [0.03644345 0.0805    ]]
VF (2):
[[0.08772662 0.04815337]
 [0.04815338 0.03299063]]
ic (2):
[[0. 0.]
 [0. 0.]

In [11]:
# --- Run the simulation for AMonly condition ---
if __name__ == '__main__':
    # You can run for a single condition to keep the output manageable, for example:
    iterative_math(condition=3, model=2)

STARTING CONDITION 3


---------- Generation 1 ----------
Omega (1):
[[0.2 0. ]
 [0.  0.3]]
Gamma (1):
[[0.29154759 0.        ]
 [0.         0.35      ]]
VY (1):
[[1. 0.]
 [0. 1.]]
mu (1):
[[0.4  0.1 ]
 [0.05 0.3 ]]
gt (1):
[[0.016 0.006]
 [0.003 0.027]]
gc (1):
[[0.016  0.0045]
 [0.0045 0.027 ]]
ht (1):
[[0.034      0.01020417]
 [0.00510208 0.03675   ]]
hc (1):
[[0.034      0.00765312]
 [0.00765312 0.03675   ]]
w (1):
[[0. 0.]
 [0. 0.]]
v (1):
[[0. 0.]
 [0. 0.]]
VF (1):
[[0. 0.]
 [0. 0.]]
ic (1):
[[0.02332381 0.00590491]
 [0.00590491 0.0315    ]]

---------- Generation 2 ----------
Omega (2):
[[0.24       0.01048625]
 [0.01366687 0.3765    ]]
Gamma (2):
[[0.34985711 0.01364893]
 [0.01780027 0.43925   ]]
VY (2):
[[1.1      0.031692]
 [0.031692 1.21675 ]]
mu (2):
[[0.36064657 0.0700001 ]
 [0.02674882 0.24379382]]
gt (2):
[[0.02113612 0.00849114]
 [0.00458915 0.03497995]]
gc (2):
[[0.02113612 0.00654015]
 [0.00654015 0.03497995]]
ht (2):
[[0.0448229  0.01439206]
 [0.00775594 0.04768494]]